In [6]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from netgraph import Graph
import statistics
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
#from umap import UMAP  
from sklearn.manifold import MDS
import seaborn as sns
from scipy.stats import pearsonr
import utils
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

Making and plotting the networks

In [3]:

party_colors = {
    100: "blue",   # Democrat
    200: "red"     # Republican
}

congresses = ['095', '096', '097','098', '099', '100', '101', '102', '103','104',
              '105', '106', '107','108', '109', '110', '111', '112','113', '114',
              '115', '116','117','118']

for congress in congresses:
    # Load edge list
    node_path = f"Data/USA/Filtered/H{congress}_filtered_USA_votes.csv"
    nodes_df = pd.read_csv(node_path)

    US_PA,US_PAV,US_PAVP,US_PP = utils.dict_create(nodes_df,'icpsr','party_code','rollnumber','cast_code')
    U_thres = np.average(utils.calc_thres(US_PP, US_PA, US_PAV))
    USA_inter= utils.calc_inter_edges(US_PP,US_PA,US_PAV)
    U_edgelist = utils.edgelist_calc(US_PA, US_PAV, U_thres)
    U_edgelist = (U_edgelist + USA_inter)
    
    df = pd.DataFrame(U_edgelist, columns=["Source", "Target"])
    df.to_csv(f"data/USA/avg_party/H{congress}_USA_edgelist.csv", index=False)

    G = nx.from_edgelist(U_edgelist)

    # Map party codes to colors using 'icpsr'
    party_dict = nodes_df.set_index("icpsr")["party_code"].to_dict()
    color_map = [party_colors.get(party_dict.get(node, None), "gray") for node in G.nodes()]

    # Generate layout
    pos = nx.spring_layout(G, seed=42)

    # Draw the network
    plt.figure(figsize=(10, 10))
    nx.draw_networkx_nodes(G, pos, node_color=color_map, node_size=30)
    nx.draw_networkx_edges(G, pos, alpha=0.3, width=0.5)
    plt.title(f"Congress {congress} Network Graph")
    plt.axis("off")

    # Add legend
    legend_handles = [
        mpatches.Patch(color="blue", label="Democrat"),
        mpatches.Patch(color="red", label="Republican"),
        mpatches.Patch(color="gray", label="Other/Unknown")
    ]
    plt.legend(handles=legend_handles, loc="lower left", fontsize="small")

    # Save plot
    out_path = f"Images/USA_Networks_avg_party/congress_{congress}_network.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close()


In [7]:

congresses = ['01_05', '05_07', '07_11', '11_15', '15_19', '19_22', '22_present']

# Fixed color mapping for all unique parties
party_colors = {
    "Alternativet": "#66c2a5",
    "Borgernes Parti": "#d53e4f",
    "Danmarksdemokraterne": "#1f78b4",
    "Dansk Folkeparti": "#a6cee3",
    "Det Konservative Folkeparti": "#33a02c",
    "Enhedslisten": "#e31a1c",
    "Folkaflokkurin": "#fb9a99",
    "Frie Grønne": "#b2df8a",
    "Inuit Ataqatigiit": "#fdbf6f",
    "Javnaðarflokkurin": "#ff7f00",
    "Kristeligt Folkeparti": "#cab2d6",
    "Kristendemokraterne": "#6a3d9a",
    "Liberal Alliance": "#ffff99",
    "Moderaterne": "#b15928",
    "Nunatta Qitornai": "#8dd3c7",
    "Ny Alliance": "#ffffb3",
    "Radikale Venstre": "#bebada",
    "Sambandsflokkurin": "#fb8072",
    "Siumut": "#80b1d3",
    "Socialdemokratiet": "#e41a1c",
    "Socialistisk Folkeparti": "#4daf4a",
    "Tjóðveldi": "#ff69b4",
    "Tjóðveldisflokkurin": "#f781bf",
    "Uden for folketingsgrupperne": "#999999",
    "Venstre": "#377eb8"
}

for congress in congresses:
    input_votes = f"Data/Denmark/Raw/P{congress}_DK.csv"

    votes_df = pd.read_csv(input_votes)
    D_PA, D_PAV, D_PAVP, D_PP = utils.dict_create(votes_df, 'aktørid','party','afstemningid','typeid_x')
    D_thres = np.average(utils.calc_thres(D_PP,D_PA,D_PAV))
    edgelist = utils.edgelist_calc(D_PA,D_PAV, D_thres)
    G = nx.from_edgelist(edgelist)

    df = pd.DataFrame(edgelist, columns=["Source", "Target"])
    #df.to_csv(f"data/Denmark/avg_party/P{congress}_DK_edgelist.csv", index=False)


    party_dict = votes_df.set_index("aktørid")["party"].to_dict()

    # Generate color map for nodes
    color_map = [party_colors.get(party_dict.get(node, None), "#cccccc") for node in G.nodes()]

    # Determine which parties are present in this graph
    present_parties = set(party_dict.get(node, None) for node in G.nodes())
    present_parties = {party for party in present_parties if party in party_colors}

    # Spring layout for visualization
    pos = nx.spring_layout(G, seed=42)

    # Draw graph
    plt.figure(figsize=(12, 12))
    nx.draw_networkx_nodes(G, pos, node_color=color_map, node_size=30)
    nx.draw_networkx_edges(G, pos, alpha=0.3, width=0.5)
    plt.title(f"Danish Parliament {congress} Network", fontsize=14)
    plt.axis("off")

    # Build legend dynamically for only the present parties
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label=party,
               markerfacecolor=party_colors[party], markersize=10)
        for party in sorted(present_parties)
    ]
    plt.legend(handles=legend_elements, title="Party", loc='lower left', fontsize='small', frameon=True)

    # Save figure
    out_path = f"Images/Denmark_avg_of_avg_networks/dk_congress_{congress}_network.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close()

